# Data Loading

In [1]:
import pandas as pd
import re
test_df = pd.read_csv("testData.csv")

# Data processing

In [ ]:
# Sort by length
index = test_df["text"].str.len().sort_values(ascending=False).index
test_df = test_df.reindex(index)

In [2]:
# Remove English alphanumeric values
alpha_pat = "[a-zA-z0-9]"

test_df["text"] = test_df["text"].str.replace(alpha_pat, "", regex=True)

In [3]:
# Handling দাঁড়ি problem
d = '।'

test_df['text'] = test_df["text"].str.replace(d, " ।", regex=True)

### Numerals

In [4]:
bangla_numerals =     ["শুন্য", "এক", "দুই", "তিন", "চার", "পাঁচ", "ছয়", "সাত", "আট", "নয়", "দশ",
                      "এগারো", "বার", "তেড়", "চৌদ্দ", "পনের", "ষোল", "সতের", "আঠার", "উনিশ", "বিশ",
                      "একুশ", "বাইশ", "তেইশ", "চব্বিশ", "পঁচিশ", "ছাব্বিশ", "সাতাশ", "আঠাশ", "ঊনত্রিশ", "ত্রিশ",
                      "একত্রিশ", "বত্রিশ", "তেত্রিশ", "চৌত্রিশ", "পঁয়ত্রিশ", "ছত্রিশ", "সাঁইত্রিশ" , "আটত্রিশ", "ঊনচল্লিশ", "চল্লিশ",
                      "একচল্লিশ", "বিয়াল্লিশ", "তেতাল্লিশ", "চুয়াল্লিশ", "পঁয়তাল্লিশ", "ছেচল্লিশ", "সাতচল্লিশ", "আটচল্লিশ", "ঊনপঞ্চাশ", "পঞ্চাশ",
                      "একান্ন", "বায়ান্ন", "তিপ্পান্ন", "চুয়ান্ন", "পঞ্চান্ন", "ছাপ্পান্ন", "সাতান্ন", "আটান্ন", "ঊনষাট", "ষাট",
                      "একষট্টি", "বাষট্টি", "তেষট্টি", "চৌষট্টি", "পঁয়ষট্টি", "ছেষট্টি", "সাতষট্টি", "আটষট্টি", "ঊনসত্তর", "সত্তর",
                      "একাত্তর", "বাহাত্তর", "তিয়াত্তর", "চুয়াত্তর", "পঁচাত্তর", "ছিয়াত্তর", "সাতাত্তর", "আটাত্তর", "ঊনআশি", "আশি",
                      "একাশি", "বিরাশি", "তিরাশি", "চুরাশি", "পঁচাশি", "ছিয়াশি", "সাতাশি", "আটাশি", "ঊননব্বই", "নব্বই",
                      "একানব্বই", "বিরানব্বই", "তিরানব্বই", "চুরানব্বই", "পঁচানব্বই", "ছিয়ানব্বই", "সাতানব্বই", "আটানব্বই", "নিরানব্বই",
                       "একশো"]

In [5]:
def num2bangla(input_number, context=False):
    if not str(input_number).isdigit():
        return "NotValid"
    #Context True Means if it is digit by digit
    if context:
        return ' '.join(bangla_numerals[int(digit)] for digit in str(input_number))

    input_number = int(input_number)

    if not 0 <= input_number <= 999999999:
        return "NotInRange"


    
    crore, input_number = divmod(input_number, 10000000)
    lakh, input_number = divmod(input_number, 100000)
    thousand, input_number = divmod(input_number, 1000)
    hundred, input_number = divmod(input_number, 100)
    tens, ones = divmod(input_number, 10)

    result = ""

    if crore:
        result += num2bangla(crore) + " কোটি "
    if lakh:
        result += num2bangla(lakh) + " লাখ"
    if thousand:
        result += " " + num2bangla(thousand) + " হাজার"
    if hundred:
        result += " " + num2bangla(hundred) + "শো"

    if (tens or ones) and input_number>=0 and input_number<=100:
        if result:
            result += " " 
        result += bangla_numerals[int(input_number)]

    return result.lstrip() or "শুন্য"

In [6]:
def replace_numerals_with_words(text, context=False):
    words = text.split()
    modified_words = []

    for word in words:
        #handled 911
        #if word == '৯১১':
            #modified_words.append(num2bangla(word, True))
            #continue
            
        #handled the plus sign
        if word == '+':
            modified_words.append('যোগ')
            continue
        if '+' in word and len(str(word))>1:
            z = 0
            y = 0
            if word[0] == '(':
                z = 1
            if word[-1] == ')':
                y = 1
            chars_to_ignore = '[()]'
            word = re.sub(chars_to_ignore, '', word)
            parts = word.split('+')
            modified_words.append(('('+num2bangla(parts[0], context) if z==1 else num2bangla(parts[0], context)))
            modified_words.append("যোগ")
            if str(parts[1]).isdigit():
                modified_words.append((num2bangla(parts[1], context)+')' if y==1 else num2bangla(parts[1], context)))
            continue
            
        #handles the equal sign
        if word == '=':
            modified_words.append('সমান')
            continue
        
        #handled date
        date = word.split('/')
        if len(date) == 3 and len(str(date[1])) == 2:
            modified_words.append(num2bangla(date[0][-2:], False))
            if date[1] == '০১':
                modified_words.append('জানুয়ারি')
            elif date[1] == '০২':
                modified_words.append('ফেব্রুয়ারি')
            elif date[1] == '০৩':
                modified_words.append('মার্চ')
            elif date[1] == '০৪':
                modified_words.append('এপ্রিল')
            elif date[1] == '০৫':
                modified_words.append('মে')
            elif date[1] == '০৬':
                modified_words.append('জুন')
            elif date[1] == '০৭':
                modified_words.append('জুলাই')
            elif date[1] == '০৮':
                modified_words.append('আগস্ট')
            elif date[1] == '০৯':
                modified_words.append('সেপ্টেম্বর')
            elif date[1] == '১০':
                modified_words.append('অক্টোবর')
            elif date[1] == '১১':
                modified_words.append('নভেম্বর')
            elif date[1] == '১২':
                modified_words.append('ডিসেম্বর')
            modified_words.append(num2bangla(date[2][:4], False))
            continue
        
        #handled double dots
        decimal = re.sub(re.escape('.'), '', word)
        if len(word.split('.')) == 3 and decimal.isnumeric():
            parts = word.split('.')
            modified_words.append(num2bangla(parts[0], False))
            modified_words.append("দশমিক")
            modified_words.append(num2bangla(parts[1], False))
            modified_words.append("দশমিক")
            modified_words.append(num2bangla(parts[2], False))
            continue
        
        #handled the "."
        decimal = re.sub(re.escape('.'), '', word)
        if '.' in word and decimal.isnumeric():
            parts = word.split('.')
            modified_words.append(num2bangla(parts[0], False))
            if str(parts[1]).isdigit():
                modified_words.append("দশমিক")
                modified_words.append(num2bangla(parts[1], True))
            continue
        
        #handle phone number
        if word.startswith('০'):
            z = 0
            if word[-1] == ',':
                z = 1
            chars_to_ignore = '[-,\.:;\'"!\?।()]'
            word = re.sub(chars_to_ignore, '', word)
            modified_words.append((num2bangla(word, context=True)+',' if z==1 else num2bangla(word, context=True)))
            continue

        #handled the ","
        comma = re.sub(re.escape(','), '', word)
        if( comma.isnumeric() and int(comma)>0):
            z = 0
            if word[-1] == ',':
                z = 1
            word = re.sub(re.escape(','), '', word);
            modified_words.append((num2bangla(word, context)+',' if z==1 else num2bangla(word, context)))
            continue
            
        #handled ১ম থেকে ১০ম
        z = ['১ম', '২য়', '৩য়', '৪র্থ', '৫ম', '৬ষ্ঠ', '৭ম', '৮ম', '৯ম', '১০ম']
        if word in z:
            if word == '১ম':
                modified_words.append('প্রথম')
                continue
            elif word == '২য়':
                modified_words.append('দ্বিতীয়')
                continue
            elif word == '৩য়':
                modified_words.append('তৃতীয়')
                continue
            elif word == '৪র্থ':
                modified_words.append('চতুর্থ')
                continue
            elif word == '৫ম':
                modified_words.append('পঞ্চম')
                continue
            elif word == '৬ষ্ঠ':
                modified_words.append('ষষ্ঠ')
                continue
            elif word == '৭ম':
                modified_words.append('সপ্তম')
                continue
            elif word == '৮ম':
                modified_words.append('অষ্টম')
                continue
            elif word == '৯ম':
                modified_words.append('নবম')
                continue
            elif word == '১০ম':
                modified_words.append('দশম')
                continue
        
        #handled ১লা থেকে ৪ঠা
        date = ['১লা', '২রা', '৩রা', '৪ঠা']
        if word in date:
            if word == '১লা':
                modified_words.append('পহেলা')
                continue
            elif word == '২রা':
                modified_words.append('দোসরা')
                continue
            elif word == '৩রা':
                modified_words.append('তেসরা')
                continue
            elif word == '৪ঠা':
                modified_words.append('চৌঠা')
                continue
        
        #handled if it is attached with a string
        else:
            temp_modified = []
            parts = re.split(r'(\d+)', word)
            for part in parts:
                if part.isnumeric():
                    part = num2bangla(part, context)
                elif part =='%':
                    part = 'শতাংশ' 
                temp_modified.append(part)
            modified_words.append(''.join(temp_modified))

    updated_text = ' '.join(modified_words)
    return updated_text

In [7]:
# Apply the 'replace_numerals_with_words' function to transform numerals in the filtered text
test_df['text'] = test_df['text'].apply(replace_numerals_with_words)

In [8]:
# Handling দাঁড়ি problem 2.0
d = ' ।'
test_df['text'] = test_df["text"].str.replace(d, "।", regex=True)

### Abbreviation

In [9]:
# ডা
d = 'ডা\.'

test_df['text'] = test_df["text"].str.replace(d, "ডাক্তার", regex=True)

In [10]:
# ড
d = 'ড\.'

test_df['text'] = test_df["text"].str.replace(d, "ডক্টর", regex=True)

In [11]:
# স.
d = '\(স\.\)'
test_df['text'] = test_df["text"].str.replace(d, "সাল্লাল্লাহু আলাইহি ওয়া সাল্লাম", regex=True)

In [12]:
# সা.
d = '\(সা\.\)'
test_df['text'] = test_df["text"].str.replace(d, "সাল্লাল্লাহু আলাইহি ওয়া সাল্লাম", regex=True)

In [13]:
# র.
d = '\(র\.\)'
test_df['text'] = test_df["text"].str.replace(d, "রাদিআল্লাহু আনহু", regex=True)

In [14]:
# রা.
d = '\(রা\.\)'
test_df['text'] = test_df["text"].str.replace(d, "রাদিআল্লাহু আনহু", regex=True)

In [15]:
# রহ.
d = '\(রহ\.\)'
test_df['text'] = test_df["text"].str.replace(d, "রহমাতুল্লাহি আলাইহি", regex=True)

In [16]:
# অব.
d = '\(অব\.\)'

test_df['text'] = test_df["text"].str.replace(d, "অবসরপ্রাপ্ত", regex=True)

In [17]:
# মো. 
d = 'মো\.'

test_df['text'] = test_df["text"].str.replace(d, "মোহাম্মদ", regex=True)

In [18]:
# মু.
d = 'মু\.'

test_df['text'] = test_df["text"].str.replace(d, "মুহম্মদ", regex=True)

In [19]:
# মুহ.
d = 'মুহ\.'

test_df['text'] = test_df["text"].str.replace(d, "মুহম্মদ", regex=True)

In [20]:
# আ.
d = 'আ\.'

test_df['text'] = test_df["text"].str.replace(d, "আব্দুল", regex=True)

In [21]:
# আওয়ামী লীগ
d = 'আ’'

test_df['text'] = test_df["text"].str.replace(d, "আওয়ামী ", regex=True)

In [22]:
test_dff = pd.read_csv("testData.csv")

In [120]:
test_df['text'][2032]

'বিস্তারিত জাতীয় বিশ্ববিদ্যালয়ের নির্দিষ্ট ওয়েবসাইট (. ... অথবা ../) থেকে জানা যাবে।'

In [121]:
test_dff['text'][2032]

'বিস্তারিত জাতীয় বিশ্ববিদ্যালয়ের নির্দিষ্ট ওয়েবসাইট ww(w. admissions.nu.edu.bd অথবা nu.edu.bd/admissions) থেকে জানা যাবে।'

In [115]:
z = "আ\’"

test_dff[lambda x: x["text"].str.contains(z)]

,row_id_column_name,text
115,115,এতে প্রধান অতিথি ছিলেন কেরানীগঞ্জ উপজেলা চেয়ার...
298,298,২৫ ইউপিতে আ’লীগের প্রতিদ্বন্দ্বী নেই : তৃতীয় ধ...
1026,1026,প্রতিমন্ত্রীর পিওর সঙ্গে ছিলেন মনিগ্রাম ইউনিয়ন...
1537,1537,এ উপজেলায় ইউপি নির্বাচনে আ’লীগের ৪১ প্রার্থী ও...
1632,1632,অভিযোগ রয়েছে- আ’লীগ সমর্থিত শাহজাদপুর পৌর মেয়র...
2049,2049,রোববার দুপুরে ওসমানীনগর উপজেলা আ’লীগের উদ্যোগে...
2122,2122,যুক্তরাজ্য আ’লীগের যুগ্ম সাধারণ সম্পাদক আনোয়ার...
2251,2251,"বাদেপাশায় আ’লীগের প্রার্থী মোস্তাক আহমদ, জাসদে..."
2273,2273,পরে উপজেলা আ’লীগের সাবেক সাধারণ সম্পাদক ও সংশ্...
2536,2536,এতে সভাপতিত্ব করেন গাঙ্গুটিয়া ইউনিয়ন আ’লীগের স...


# Model

In [ ]:
from transformers import pipeline
model_id = "iit-eight/checkpoint-36000/"
pipe = pipeline("text2text-generation", model=model_id, device=0)

# Inference

In [ ]:
%%time
texts = test_df["text"].tolist()
ipas = pipe(texts, max_length=128, batch_size=16)
ipas = [ipa["generated_text"] for ipa in ipas]

In [ ]:
ipas

In [ ]:
test_df

In [ ]:
test_df["ipa"] = ipas
test_df = test_df.sort_index()
test_df.head()

In [ ]:
test_df = test_df.drop('text', axis = 1)

In [ ]:
test_df.to_csv("submission_jup_eightt_one.csv", index = None)